In [1]:
import os
import re
import sys
print(os.path.exists('/media/athira/CVlab_IITM/CV lab documents/Works/My works/Openpose/pytorch_Realtime_Multi-Person_Pose_Estimation/'))
sys.path.append('.')
#sys.path.append('../')
#sys.path.insert(0, '/media/athira/CVlab_IITM/CV lab documents/Works/My works/Openpose/pytorch_Realtime_Multi-Person_Pose_Estimation/')
import cv2
import math
import time
import scipy
import argparse
import matplotlib
import numpy as np
import pylab as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from collections import OrderedDict
from scipy.ndimage.filters import gaussian_filter
#import network.rtpose_vgg
from network.rtpose_vgg import get_model
from network.post import decode_pose
from training.datasets.coco_data.preprocessing import (inception_preprocess,
                                              rtpose_preprocess,
                                              ssd_preprocess, vgg_preprocess)
from network import im_transform
from evaluate.coco_eval import get_multiplier, get_outputs, handle_paf_and_heat
torch.cuda.set_device(0)

True


In [2]:
model = get_model('vgg19')     
model = torch.nn.DataParallel(model).cuda()
weight_name = './network/weight/pose_model_scratch.pth'
model.load_state_dict(torch.load(weight_name))
model = model.module

model.eval()
model.float()
model.cuda()
#torch.cuda.empty_cache()


Bulding VGG19


rtpose_model(
  (model0): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace)
    (16): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (17): ReLU(i

In [3]:
torch.load(weight_name)

OrderedDict([('module.model0.0.weight', tensor([[[[-0.0308, -0.0271, -0.0306],
                        [ 0.0354,  0.0478,  0.0099],
                        [ 0.0228, -0.0034, -0.0129]],
              
                       [[ 0.0125,  0.0654,  0.0226],
                        [ 0.1446,  0.2554,  0.1573],
                        [ 0.0761,  0.1789,  0.0444]],
              
                       [[-0.0086,  0.0849,  0.0590],
                        [ 0.0653,  0.1847,  0.0805],
                        [-0.0542,  0.0279, -0.0898]]],
              
              
                      [[[ 0.3031, -0.4052, -0.6759],
                        [ 0.5876, -0.3192, -0.6728],
                        [ 0.7972,  0.5729, -0.1306]],
              
                       [[ 0.2663, -0.4047, -0.5780],
                        [ 0.4723, -0.3004, -0.5616],
                        [ 0.6326,  0.5706, -0.1382]],
              
                       [[-0.0156, -0.0948, -0.0544],
                        [ 0.06

In [4]:
import os 
imgs_path='/media/athira/CVlab_IITM/CV lab documents/Works/My works/Openpose/pytorch_Realtime_Multi-Person_Pose_Estimation/demo/scratch'
img_names=os.listdir(imgs_path)
img_names.sort()
#torch.cuda.empty_cache()
img_names

['0001C1T0001F001.jpg',
 '0001C1T0001F002.jpg',
 '0001C1T0001F003.jpg',
 '0001C1T0001F004.jpg',
 '0001C1T0001F005.jpg',
 '0001C1T0001F006.jpg',
 '0001C1T0001F007.jpg',
 '0001C1T0001F008.jpg',
 '0001C1T0001F009.jpg',
 '0001C1T0001F010.jpg',
 '0001C1T0001F011.jpg',
 '0001C1T0001F012.jpg',
 '0001C1T0001F013.jpg',
 '0001C1T0001F014.jpg',
 '0001C1T0001F015.jpg',
 '0001C1T0001F016.jpg',
 '0001C1T0002F001.jpg',
 '0001C1T0002F002.jpg',
 '0001C1T0002F003.jpg',
 '0001C1T0002F004.jpg',
 '0001C1T0002F005.jpg']

In [ ]:
for im_name in img_names:
    test_image = os.path.join(imgs_path,im_name)
    oriImg = cv2.imread(test_image) # B,G,R order
    shape_dst = np.min(oriImg.shape[0:2])

    # Get results of original image
    multiplier = get_multiplier(oriImg)

    with torch.no_grad():
        orig_paf, orig_heat = get_outputs(
            multiplier, oriImg, model,'rtpose')
          
        # Get results of flipped image
        swapped_img = oriImg[:, ::-1, :]
        flipped_paf, flipped_heat = get_outputs(multiplier, swapped_img,
                                                model,'rtpose')

        # compute averaged heatmap and paf
        paf, heatmap = handle_paf_and_heat(
            orig_heat, flipped_heat, orig_paf, flipped_paf)
            
    param = {'thre1': 0.1, 'thre2': 0.05, 'thre3': 0.5}
    canvas, to_plot, candidate, subset = decode_pose(
        oriImg, param, heatmap, paf)
    
    
    cv2.imwrite('./demo/scratch/result_'+  im_name + '.png',to_plot)
    torch.cuda.empty_cache()

#     cv2.imwrite("result_%d.jpg" %im_name, image)
#     cv2.imwrite('resultNew.png',to_plot)


